In [167]:
## HW3 DATSCI W261 

**Names** Safyre Anderson, Howard Wen , Vamsi Sakhamuri

**Emails** safyre@berkelye.edu, howard.wen1@gmail.com, vamsi@ischool.berkeley.edu 

**Time of Initial Submission:** February 4nd, 2016 8am PST

**Section** W261-3, Spring 2016  

**Week** 3 Homework

### HW 3.0.
*What is a merge sort? Where is it used in Hadoop?*

*How is  a combiner function in the context of Hadoop?* 

*Give an example where it can be used and justify why it should be used in the context of this problem.*

*What is the Hadoop shuffle?*


### HW 3.0.1
*What is a merge sort? Where is it used in Hadoop?*

* Merge-sort is a sorting algorithm which runs in O(nlog(n)) time. In hadoop, it is used in between the map outputs and 
the reduce inputs. Output from each map output is sorted by the key and then merged with the other sorted outputs from other 
map outputs.

<img src="map-reduce.png">

### HW 3.0.2

*How is  a combiner function in the context of Hadoop?* 

*Give an example where it can be used and justify why it should be used in the context of this problem.*

* Combiner function is used after the map output is generated. The values from the same keys are then aggregated to reduce network communication in their transfer to the reduce nodes. Network communication can be a bottleneck and a combiner helps alleviate this problem. 
Whether a combiner function is run or not is entirely dependent on the execution framework. To be totally sure that intermediate map outputs are combined, in-mapper combining function can be employed. This ensures that aggregation will happen 100% before being shuffled across the network to the various reduce nodes.

* An example where combiners (either the optional one provided by the execution framework or the in-mapper combiner) is helpful would be the word count application.
Without any sort of combiners, we would be transfer the same key multiple times within the same map task.

    For example, if the map input is "hello hello hi hi hello hello", 

    Without combiners, the following <key,value> pairs will be emitted across the network and into the reduce nodes:

    hello,1
    
    hello,1
    
    hi,1
    
    hi,1
    
    hello,1
    
    hello,1

    With combiners, the following <key,value> pairs will be emitted across the network and into the reduce nodes:

    hello,4
    
    hi,2

    This is an incredible reduction in network traffic.
    

<img src='hadoop-tutorial-mapreduce-part-4-input-and-output-31-638.jpg'>

### HW 3.0.3

*What is the Hadoop shuffle?*

* Hadoop Shuffle: Map-reduce makes the guarantee that the input to every reducer is sorted by the key. And that the <key,value> pairs with the same key are routed to the same reducer. The process by which the system performs this sort and merge (across various mappers) and transfers the map outputs to the reducers is known as the hadoop shuffle.


<img src="shuffle.png">

### HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress)
*Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 
While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.
Use the Consumer Complaints  Dataset provide here to complete this question:*
    

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

*The consumer complaints dataset consists of diverse consumer complaints, 
which have been reported across the United States regarding various types of loans. 
The dataset consists of records of the form:*

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

*Here’s is the first few lines of the  of the Consumer Complaints  Dataset:*

`Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,`


In [168]:
#Start hdfs

!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/start-dfs.sh

16/02/04 08:14:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: namenode running as process 1498. Stop it first.
localhost: datanode running as process 1042. Stop it first.
Starting secondary namenodes [0.0.0.0]
0.0.0.0: secondarynamenode running as process 1162. Stop it first.
16/02/04 08:14:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [169]:
#start the jobtracker

!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/mr-jobhistory-daemon.sh --config /Users/Vamsi/Downloads/hadoop-2.7.1/etc/hadoop/ start historyserver 

historyserver running as process 1275. Stop it first.


In [170]:
#Service check
!jps

1042 DataNode
1162 SecondaryNameNode
1498 NameNode
1275 JobHistoryServer
4719 Jps


In [171]:
#Start afresh
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f  /user/hw3

16/02/04 08:14:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:14:57 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hw3


In [172]:
#Create a directory for hw3
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -mkdir -p /user/hw3

16/02/04 08:14:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [173]:
#Load the dataset for 3.1
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put Consumer_Complaints.csv /user/hw3

16/02/04 08:15:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [174]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
#Grabbing the header names
header = next(csv_iter)

for line in csv_iter:
    if(len(line)==len(header)):   #continue only if all the rows have all the entries
        if(line[1].lower()=="debt collection"):
            sys.stderr.write("reporter:counter:Product,Debt,1\n")
        elif(line[1].lower() == "mortgage"):
            sys.stderr.write("reporter:counter:Product,Mortgage,1\n")
        else:
            sys.stderr.write("reporter:counter:Product,Other,1\n")

Overwriting mapper.py


In [175]:
!chmod a+x mapper.py

In [176]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/Consumer_Complaints.csv \
-output /user/hw3/output_3_1 \
-mapper mapper.py \
-reducer org.apache.hadoop.mapred.lib.IdentityReducer 

16/02/04 08:15:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:15:08 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:15:08 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:15:08 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:15:09 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:15:09 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:15:09 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:15:09 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local12979890_0001
16/02/04 08:15:09 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/04 08:15:09 INFO mapred.LocalJobRunner: OutputCom

<img src="counters.png">

### HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters

*For this brief study the Input file will be one record (the next line only):*

`foo foo quux labs foo bar quux`


### 3.2.1 

*Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.*

In [177]:
#Create a file 
!echo "foo foo quux labs foo bar quux" >input_3_2

In [178]:
#Load the file into hdfs
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put input_3_2 /user/hw3

16/02/04 08:15:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [179]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")   #trigger counter

for line in sys.stdin:
    line_s = re.split(r'[\s]',line)   #split on whitespace
    for l in line_s:
        print "%s\t1" %l.strip()    #Emit word,1 

Overwriting mapper.py


In [180]:
!chmod a+x mapper.py

In [181]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")  #trigger counter

for line in sys.stdin:
    line_s = re.split(r'[\t]',line)   #split on tab
    
    if((prev_string!=None) and (prev_string != line_s[0])): # Check whether the a new key is emitted
        print prev_string,count  # If a new key is emitted then commit the previous key and count to disk
        count = 0   #reset the counter

    count += 1    #increment the counter
    prev_string = line_s[0] 
print prev_string,count  

Overwriting reducer.py


In [182]:
!chmod a+x reducer.py

In [183]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/hw3/input_3_2 \
-output /user/hw3/output_3_2_1 \
-mapper mapper.py \
-reducer reducer.py 

16/02/04 08:15:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:15:20 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:15:20 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:15:20 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:15:20 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:15:20 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:15:20 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:15:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local167321748_0001
16/02/04 08:15:21 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/04 08:15:21 INFO mapred.LocalJobRunner: OutputCo

<img src="counters_3_2.png">

The number of mapper calls is 1 because the file is small enough to be processed by just 1 mapper.
The number of reducer calls is 4 because there are 4 unique words and 4 reducers are used.

** Following 4 cells would output each of the reducer outputs **

In [184]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_1/part-00000

16/02/04 08:15:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
quux 2	


In [185]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_1/part-00001 

16/02/04 08:15:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
 1	
foo 3	


In [186]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_1/part-00002 

16/02/04 08:15:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bar 1	


In [187]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_1/part-00003

16/02/04 08:15:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
labs 1	


### 3.2.2 

*Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).*
*Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.*

In [188]:
#Removing the header from the Consumer_Complaints.csv file
!tail -n +2 Consumer_Complaints.csv > Consumer_Complaints_no_header.csv

In [189]:
#Split the file into 4 equally sized files
!split -l 78228 Consumer_Complaints_no_header.csv

In [190]:
#Load the splitted files into hdfs
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put xa* /user/hw3

16/02/04 08:15:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [191]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3].strip())      #splitting the issues column
        for w in issues:
            print "%s\t1" %w #Emit word,1 to the reducer 

Overwriting mapper.py


In [192]:
!chmod a+x mapper.py

In [193]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")
    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0

    count += 1
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [194]:
!chmod a+x reducer.py

In [195]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f /user/hw3/output_3_2_2

16/02/04 08:15:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [196]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/hw3/xa* \
-output /user/hw3/output_3_2_2 \
-mapper mapper.py \
-reducer reducer.py 

16/02/04 08:15:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:15:49 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:15:49 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:15:49 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:15:49 INFO mapred.FileInputFormat: Total input paths to process : 4
16/02/04 08:15:49 INFO mapreduce.JobSubmitter: number of splits:4
16/02/04 08:15:49 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:15:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local983135479_0001
16/02/04 08:15:50 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/04 08:15:50 INFO mapred.LocalJobRunner: OutputCo

<img src="3_2_2.png">

** The next four cells output the first 10 rows of each of the 4 reducer outputs **

In [197]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00000 | head

16/02/04 08:16:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Account 16555	
Applied 139	
Can't 1999	
Cash 240	
Closing 2795	
Cont'd 17972	
Debt 1343	
Delinquent 1061	
I 925	
Incorrect 29133	


In [198]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00001 | head

16/02/04 08:16:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
1 4	
ATM 2422	
Cancelling 2795	
Communication 8671	
Dealing 1944	
Improper 4966	
Loan 107254	
Payment 92	
Problems 9484	
Shopping 672	


In [199]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00002 | head

16/02/04 08:16:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
APR 3431	
Arbitration 168	
Bankruptcy 222	
Billing 8158	
Convenience 75	
Credit 14768	
Deposits 10555	
Disclosure 7655	
False 3621	
Overlimit 127	


In [200]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00003 | head

16/02/04 08:16:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Advertising 1193	
Application 8868	
Balance 597	
Charged 878	
Collection 1907	
Customer 2734	
Embezzlement 3276	
Forbearance 350	
Fraud 3842	
Getting 291	


### 3.2.3 

*Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.* 

In [201]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3])
        for w in issues:
            print "%s\t1" %w

Overwriting mapper.py


In [202]:
!chmod a+x mapper.py

In [203]:
%%writefile combiner.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:COMBINER,combiner_calls,1\n")
    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print "%s\t%s" %(prev_string,count)
        count = 0
    count += 1
    prev_string = line_s[0]
print "%s\t%s" %(prev_string,count)

Overwriting combiner.py


In [204]:
!chmod a+x combiner.py

In [205]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")

    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0
    count += int(line_s[1])
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [206]:
!chmod a+x reducer.py

In [207]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f /user/hw3/output_3_2_3

16/02/04 08:16:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [208]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/hw3/xa* \
-output /user/hw3/output_3_2_3 \
-mapper mapper.py \
-combiner combiner.py \
-reducer reducer.py 

16/02/04 08:16:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:16:33 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:16:33 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:16:33 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:16:34 INFO mapred.FileInputFormat: Total input paths to process : 4
16/02/04 08:16:34 INFO mapreduce.JobSubmitter: number of splits:4
16/02/04 08:16:34 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:16:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local2063302883_0001
16/02/04 08:16:34 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/04 08:16:34 INFO mapred.LocalJobRunner: OutputC

<img src="3_2_3.png">

The power of combiners is evident above. The reducer_calls reduces to 661!

### 3.2.4 

*Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items).* 

In [209]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3])
        for w in issues:
            print "%s,1" %w
            print "*,1"

Overwriting mapper.py


In [210]:
!chmod a+x mapper.py

In [211]:
%%writefile combiner.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

for line in sys.stdin:
    line_s = re.split(r'[,]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print "%s,%s" %(prev_string,count)
        count = 0
    count += 1
    prev_string = line_s[0]
print "%s,%s" %(prev_string,count)

Overwriting combiner.py


In [212]:
!chmod a+x combiner.py

In [213]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    line_s = re.split(r'[,]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0
    count += int(line_s[1])
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [214]:
!chmod a+x reducer.py

In [215]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f /user/hw3/output_3_2_4

16/02/04 08:16:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [216]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/xa* \
-output /user/hw3/output_3_2_4 \
-mapper mapper.py \
-combiner combiner.py \
-reducer reducer.py 

16/02/04 08:17:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:17:02 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:17:02 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:17:02 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:17:02 INFO mapred.FileInputFormat: Total input paths to process : 4
16/02/04 08:17:03 INFO mapreduce.JobSubmitter: number of splits:4
16/02/04 08:17:03 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:17:03 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1641761722_0001
16/02/04 08:17:03 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/04 08:17:03 INFO mapreduce.Job: The url to track the 

In [217]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_4/part-00000 | head

16/02/04 08:17:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
* 1348312	
 4	
APR 3431	
ATM 2422	
Account 16555	
Advertising 1193	
Application 8868	
Applied 139	
Arbitration 168	
Balance 597	


In [218]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_4/part-00000 > reduce_3_2_4

16/02/04 08:17:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


** Running a second map-reduce job to sort the word counts **

In [219]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

for line in sys.stdin:
    line = re.split(r'[\s]+',line)
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [220]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
i=0
for line in sys.stdin:
    line = re.split(r'[,]',line.strip())

    if(i==0):
        total_count = int(line[1])
        total_count_read = 1
        i = i+1
    if(total_count_read==1):
        rel_freq = float(line[1])/float(total_count)
        print "%s,%s,%s,%s" %(line[0],line[1],rel_freq,total_count)

Overwriting reducer.py


In [221]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put reduce_3_2_4 /user/hw3

16/02/04 08:17:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [222]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f /user/hw3/output_3_2_4f

16/02/04 08:17:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [223]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=2 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k2,2nr \
-D mapred.reduce.tasks=1 \
-input /user/hw3/reduce_3_2_4 \
-output /user/hw3/output_3_2_4f \
-mapper mapper.py \
-reducer reducer.py

16/02/04 08:17:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:17:35 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:17:35 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:17:35 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:17:35 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:17:35 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:17:36 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/04 08:17:36 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/04 08:17:36 INFO Configuration.deprecation: mapr

** Printing out the 50 most frequent words alongwith their frequencies and relative frequencies below. **

The format of the output is:

Word, Frequency , Relative Frequency, Total Count of words

In [224]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_4f/part-00000 | head -n 51

16/02/04 08:17:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
*,1348312,1.0,1348312	
Loan,107254,0.0795468704573,1348312	
foreclosure,70487,0.0522779594041,1348312	
collection,70487,0.0522779594041,1348312	
modification,70487,0.0522779594041,1348312	
account,40893,0.0303290336361,1348312	
or,40508,0.0300434914174,1348312	
credit,40483,0.0300249497149,1348312	
payments,39993,0.0296615323456,1348312	
escrow,36767,0.0272689110532,1348312	
servicing,36767,0.0272689110532,1348312	
report,34903,0.0258864417138,1348312	
Incorrect,29133,0.0216070167736,1348312	
on,29069,0.0215595500151,1348312	
information,29069,0.0215595500151,1348312	
debt,26531,0.0196771963759,1348312	
not,18477,0.013703801494,1348312	
owed,17972,0.0133292591032,1348312	
collect,17972,0.0133292591032,1348312	
attempts,17972,0.0133292591032,1348312	
Cont'd,17972,0.0133292591032,1348312	
Account,16555,0.0122783154047,1348312	
and,16448,0.012

In [225]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_4f/part-00000 | tail -n 10

16/02/04 08:17:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
credited,92,6.82334652514e-05,1348312	
Payment,92,6.82334652514e-05,1348312	
Convenience,75,5.56251075419e-05,1348312	
checks,75,5.56251075419e-05,1348312	
wrong,71,5.26584351396e-05,1348312	
day,71,5.26584351396e-05,1348312	
amt,71,5.26584351396e-05,1348312	
missing,64,4.74667584357e-05,1348312	
disclosures,64,4.74667584357e-05,1348312	
,4,2.96667240223e-06,1348312	


### HW3.3. Shopping Cart Analysis
*Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.*

*For this homework use the online browsing behavior dataset located at:* 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

*Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.*

*Here are the first few lines of the ProductPurchaseData* 

`FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444` 


**Do some exploratory data analysis of this dataset.** 

*How many unique items are available from this supplier?*

*Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce.*

In [226]:
#Loading the data-set
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put ProductPurchaseData.txt /user/hw3

16/02/04 08:17:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [227]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    line = re.split(r'[\s]',line.strip())
    for l in line:
        print "%s\t1" %l
        print "*\t1"

Overwriting mapper.py


In [228]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

count = 0
prev_id = None

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    if((prev_id !=None) and (line[0] !=prev_id)):
        print "%s\t%s" %(prev_id,count)
        count = 0
    count +=1
    prev_id = line[0]
print "%s\t%s" %(prev_id,count)

Overwriting reducer.py


In [229]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/ProductPurchaseData.txt \
-output /user/hw3/output_3_3 \
-mapper mapper.py \
-reducer reducer.py 

16/02/04 08:17:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:17:47 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:17:47 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:17:47 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:17:48 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:17:48 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:17:48 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:17:48 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local38036051_0001
16/02/04 08:17:48 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/04 08:17:48 INFO mapreduce.Job: The url to track the jo

In [230]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_3/part-00000 > counts_3_3

16/02/04 08:17:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [231]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put counts_3_3 /user/hw3

16/02/04 08:18:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


** Using a second map-reduce job to perform the sort **

In [232]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

i = 0
for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [233]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
i=0
for line in sys.stdin:
    line = re.split(r'[,]',line.strip())

    if(i==0):
        total_count = int(line[1])
        i = i+1
    elif(i==1):
        rel_freq = float(line[1])/float(total_count)
        print "%s,%s,%s,%s" %(line[0],line[1],rel_freq,total_count)


Overwriting reducer.py


In [234]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=2 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k2,2nr \
-D mapred.reduce.tasks=1 \
-input /user/hw3/counts_3_3 \
-output /user/hw3/output_3_3_out \
-mapper mapper.py \
-reducer reducer.py

16/02/04 08:18:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:18:03 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:18:03 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:18:03 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:18:03 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:18:03 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:18:04 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/04 08:18:04 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/04 08:18:04 INFO Configuration.deprecation: mapr

In [235]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_3_out/part-00000 | wc -l 

16/02/04 08:18:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
   12592


** The number of unique items is 12592 **

** The output below is the top 50 most encountered product ID's alongwith their frequencies and relative frequencies **

** The format is Product ID, Frequency, Relative Frequency, Total Number of ID's **

In [236]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_3_out/part-00000 | head -n 50

16/02/04 08:18:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779,6667,0.0175067747831,380824	
FRO40251,3881,0.010191059387,380824	
ELE17451,3875,0.0101753040775,380824	
GRO73461,3602,0.00945843749344,380824	
SNA80324,3044,0.00799319370628,380824	
ELE32164,2851,0.0074863979161,380824	
DAI75645,2736,0.00718442114993,380824	
SNA45677,2455,0.0064465474865,380824	
FRO31317,2330,0.0061183118711,380824	
DAI85309,2293,0.00602115412894,380824	
ELE26917,2292,0.00601852824402,380824	
FRO80039,2233,0.00586360103355,380824	
GRO21487,2115,0.00555374661261,380824	
SNA99873,2083,0.00546971829507,380824	
GRO59710,2004,0.00526227338613,380824	
GRO71621,1920,0.00504169905258,380824	
FRO85978,1918,0.00503644728273,380824	
GRO30386,1840,0.00483162825872,380824	
ELE74009,1816,0.00476860702057,380824	
GRO56726,1784,0.00468457870302,380824	
DAI63921,1773,0.00465569396887,380824	
GRO46854,1756,0.00461105392517,380824	
E

### HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

*Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.*

*List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support ( the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2.* 

*Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.* 

*Please output records of the following form for the top 50 pairs (itemsets of size 2):*

      *item1, item2, support count, support*

*Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order.* 

*Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.*

In [237]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:group,Num mapper calls,1\n")

for line in sys.stdin:
    line = re.split(r'[\s]',line.strip())
    for l in line:
        for p in line:
            if(p > l):
                print "%s,%s\t1" %(l,p)
            elif (l > p):
                 print "%s,%s\t1" %(p,l)               
                #print "%s,*\t1" %l   Use this only if you want total number of tuples in the denominator for support
    print "BASKET,*\t1" #Use this if you want basket in the denominator for support calculation

Overwriting mapper.py


In [238]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

count = 0
prev_id = None

sys.stderr.write("reporter:counter:group,Num reducer calls,1\n")

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    if((prev_id !=None) and (line[0] !=prev_id)):
        print "%s\t%s" %(prev_id,count)
        count = 0
    count +=1
    prev_id = line[0]
print "%s\t%s" %(prev_id,count)

Overwriting reducer.py


In [239]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f /user/hw3/output_3_4i

16/02/04 08:18:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [240]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/ProductPurchaseData.txt \
-output /user/hw3/output_3_4i \
-mapper mapper.py \
-reducer reducer.py 

16/02/04 08:18:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:18:15 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:18:15 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:18:15 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:18:16 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:18:16 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:18:16 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:18:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1172650280_0001
16/02/04 08:18:16 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/04 08:18:16 INFO mapred.LocalJobRunner: OutputC

In [241]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_4i/part-00000 > pairs_unsorted

16/02/04 08:19:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [242]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put pairs_unsorted /user/hw3

16/02/04 08:19:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**This second Map-reduce job is used to sort the pairs**

In [243]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

sys.stderr.write("reporter:counter:group,Num mapper calls,1\n")

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [244]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:group,Num reducer calls,1\n")

for line in sys.stdin:
    line = re.split(r'[,]',line.strip())
    if(line[1]=="*"):
        total = int(line[2]) 
    else:
        rel_freq = float(line[2])/float(total)
        print "%s,%s,%s,%s,%s" %(line[0],line[1],line[2],rel_freq,total)

Overwriting reducer.py


In [245]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=3 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k3,3nr \
-D mapred.reduce.tasks=1 \
-input /user/hw3/pairs_unsorted \
-output /user/hw3/output_3_4_out \
-mapper mapper.py \
-reducer reducer.py

16/02/04 08:19:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:19:06 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:19:06 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:19:06 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:19:06 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:19:06 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:19:06 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/04 08:19:06 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/04 08:19:06 INFO Configuration.deprecation: mapr

** Computational Setup : **
    
    MacBook Air
    Processor : 1.8 Ghz Intel Core i5 , 2 Cores
    Memory : 4 GB 1600 Mhz DDR3
    Disk : SSD (Flash Storage) 128 GB

*The output is shown below*

*Format = Pairs, Support Count, Support, Total number of baskets *

In [246]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_4_out/part-00000 | head -n 50

16/02/04 08:19:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779,ELE17451,3184,0.102376129385,31101	
FRO40251,SNA80324,2824,0.0908009388766,31101	
DAI75645,FRO40251,2508,0.0806404938748,31101	
FRO40251,GRO85051,2426,0.0780039227035,31101	
DAI62779,GRO73461,2278,0.0732452332722,31101	
DAI75645,SNA80324,2260,0.0726664737468,31101	
DAI62779,FRO40251,2140,0.0688080769107,31101	
DAI62779,SNA80324,1846,0.0593550046622,31101	
DAI62779,DAI85309,1836,0.0590334715926,31101	
ELE32164,GRO59710,1822,0.058583325295,31101	
FRO40251,GRO73461,1764,0.0567184334909,31101	
DAI62779,DAI75645,1764,0.0567184334909,31101	
DAI62779,ELE92920,1754,0.0563969004212,31101	
FRO40251,FRO92469,1670,0.0536960226359,31101	
DAI62779,ELE32164,1664,0.0535031027941,31101	
DAI75645,GRO73461,1424,0.0457863091219,31101	
DAI43223,ELE32164,1422,0.045722002508,31101	
DAI62779,GRO30386,1418,0.0455933892801,31101	
ELE17451,FRO40251,1394,0.04

### HW3.5: Stripes
*Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.*

*Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)*

*Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs*

In [247]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:group,Num mapper calls,1\n")

for line in sys.stdin:
    line = re.split(r'[\s]',line.strip())
    
    for l in line:
        co_stripe = {}
        for p in line:
            if(l != p):
                if p not in co_stripe.keys():
                    co_stripe[p] = 1 
                else:
                    co_stripe[p] += 1
        print "%s\t%s" %(l,co_stripe)
                #print "%s,*\t1" %l   Use this only if you want total number of tuples in the denominator for support
    print "BASKET\t{'*':1}" #Use this if you want basket in the denominator for support calculation

Overwriting mapper.py


In [248]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import ast

count = {}
prev_id = None

sys.stderr.write("reporter:counter:group,Num mapper calls,1\n")

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    co_strip = ast.literal_eval(line[1])
    if((prev_id !=None) and (line[0] !=prev_id)):
        for w in count.keys():
            if(prev_id < w):
                print "%s,%s\t%s" %(prev_id,w,count[w])
            elif(prev_id >w):
                print "%s,%s\t%s" %(w,prev_id,count[w])                
        count = {}
    for w in co_strip:
        if w in count.keys():
            count[w] +=1
        else:
            count[w] = 1
    prev_id = line[0]

for w in count.keys():
    if(prev_id <w):
        print "%s,%s\t%s" %(prev_id,w,count[w])
    elif(prev_id >w):
        print "%s,%s\t%s" %(w,prev_id,count[w])         

Overwriting reducer.py


In [249]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/ProductPurchaseData.txt \
-output /user/hw3/output_3_5i \
-mapper mapper.py \
-reducer reducer.py 

16/02/04 08:19:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:19:27 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:19:27 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:19:27 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:19:27 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:19:28 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:19:28 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:19:28 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local112720032_0001
16/02/04 08:19:28 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/04 08:19:28 INFO mapreduce.Job: Running job: job

In [250]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_5i/part-00000 > stripes_unsorted

16/02/04 08:22:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [251]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put stripes_unsorted /user/hw3

16/02/04 08:23:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


** Running a second map-reduce job to merge same pairs **

**For example if there are two keys**

**A:{'B':2,'C':1}**

**B:{'A':1,'C':1}**

**The first map reduce job would output the following**

**A,B 2**

**A,B 1**

**etc...**

**The second map reduce job would merge these two pairs to give**

**A,B 3**

**And the third map reduce job is to sort the counts!**

In [252]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:group,Num mapper calls,1\n")

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    print "%s\t%s" %(line[0],line[1])

Overwriting mapper.py


In [253]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:group,Num mapper calls,1\n")

prev_id = None
count = 0
for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    if((prev_id !=None) and (line[0] !=prev_id)):
        print "%s\t%s" %(prev_id,count)
        count = 0
    count += int(line[1])
    prev_id = line[0]
print "%s\t%s" %(prev_id,count)

Overwriting reducer.py


In [254]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/stripes_unsorted \
-output /user/hw3/stripes_unsorted_int \
-mapper mapper.py \
-reducer reducer.py

16/02/04 08:23:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:23:07 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:23:07 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:23:07 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:23:08 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:23:08 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:23:08 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/04 08:23:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1940809252_0001
16/02/04 08:23:08 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/04 08:23:08 INFO mapred.LocalJobRunner: OutputC

In [255]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/stripes_unsorted_int/part-00000 > stripes_unsorted_int

16/02/04 08:23:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [256]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f /user/hw3/stripes_unsorted_int

16/02/04 08:23:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:23:41 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hw3/stripes_unsorted_int


In [257]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put stripes_unsorted_int /user/hw3

16/02/04 08:23:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**Third Map reduce job to perform the sort**

In [258]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:group,Num mapper calls,1\n")

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [259]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:group,Num mapper calls,1\n")

for line in sys.stdin:
    line = re.split(r'[,]',line.strip())
    if(line[0]=="*"):
        total = int(line[2]) 
    else:
        rel_freq = float(line[2])/float(total)
        print "%s,%s,%s,%s,%s" %(line[0],line[1],line[2],rel_freq,total)

Overwriting reducer.py


In [260]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=3 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k3,3nr \
-D mapred.reduce.tasks=1 \
-input /user/hw3/stripes_unsorted_int \
-output /user/hw3/output_3_5_o \
-mapper mapper.py \
-reducer reducer.py

16/02/04 08:23:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 08:23:46 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 08:23:46 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 08:23:46 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 08:23:47 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 08:23:47 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 08:23:47 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/04 08:23:47 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/04 08:23:47 INFO Configuration.deprecation: mapr

** Computational Setup : **
    
    MacBook Air
    Processor : 1.8 Ghz Intel Core i5 , 2 Cores
    Memory : 4 GB 1600 Mhz DDR3
    Disk : SSD (Flash Storage) 128 GB

*The output is shown below*

*Format = Pairs, Support Count, Support, Total number of baskets *

In [261]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_5_o/part-00000 | head -n 50

16/02/04 08:24:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779,ELE17451,3184,0.102376129385,31101	
FRO40251,SNA80324,2824,0.0908009388766,31101	
DAI75645,FRO40251,2508,0.0806404938748,31101	
FRO40251,GRO85051,2426,0.0780039227035,31101	
DAI62779,GRO73461,2278,0.0732452332722,31101	
DAI75645,SNA80324,2260,0.0726664737468,31101	
DAI62779,FRO40251,2140,0.0688080769107,31101	
DAI62779,SNA80324,1846,0.0593550046622,31101	
DAI62779,DAI85309,1836,0.0590334715926,31101	
ELE32164,GRO59710,1822,0.058583325295,31101	
DAI62779,DAI75645,1764,0.0567184334909,31101	
FRO40251,GRO73461,1764,0.0567184334909,31101	
DAI62779,ELE92920,1754,0.0563969004212,31101	
FRO40251,FRO92469,1670,0.0536960226359,31101	
DAI62779,ELE32164,1664,0.0535031027941,31101	
DAI75645,GRO73461,1424,0.0457863091219,31101	
DAI43223,ELE32164,1422,0.045722002508,31101	
DAI62779,GRO30386,1418,0.0455933892801,31101	
ELE17451,FRO40251,1394,0.04

In [262]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/stop-dfs.sh

16/02/04 08:24:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/02/04 08:24:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
